In [1]:
import re
import string
import warnings
import datetime

import numpy as np
import pandas as pd

from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

warnings.filterwarnings('ignore')

# Data Cleaning

In [2]:
df_sbuzz = pd.read_csv("syndata/Box Office/Recruitment x Box Office x Q1 2019 - Twitter.csv", sep="\t")
df_title = pd.read_csv("syndata/Box Office/Recruitment x Box Office x Q1 2019 - Titles Performace.csv", sep="\t")

In [3]:
df_sbuzz = df_sbuzz.drop(axis=1, labels=['Unnamed: 0'])

df_sbuzz['title'] = df_sbuzz['title'].str.lower()
df_title['Title'] = df_title['Title'].str.lower()

df_title['Title'] = df_title['Title'].apply(lambda x: re.sub("[^\w\s]", "", x))
df_title['Title'] = df_title['Title'].apply(lambda x: re.sub("avengers infinty war", "avengers infinity war", x))

df_sbuzz['title'] = df_sbuzz['title'].apply(lambda x: re.sub("[^\w\s]", "", x))
df_sbuzz['title'] = df_sbuzz['title'].apply(lambda x: re.sub("predator", "the predator", x))

df_sbuzz['post_date'] = pd.to_datetime([datetime.datetime.strptime(x[0], '%Y-%m-%d').date() for x in list(df_sbuzz['post_date_time'].str.split('T'))])
df_sbuzz['post_time'] = [datetime.datetime.strptime(x[1].split('.')[0], '%H:%M:%S').time() for x in list(df_sbuzz['post_date_time'].str.split('T'))]

In [4]:
movie_dict = {}
c = 0

for i in df_sbuzz.index:
    if df_sbuzz.at[i, 'title'] not in movie_dict:
        c+=1
        movie_dict[df_sbuzz.at[i, 'title']] = c

for i in df_title.index:
    if df_title.at[i, 'Title'] not in movie_dict:
        c+=1
        movie_dict[df_title.at[i, 'Title']] = c
        
rev_movie_dict = {v:k for k, v in movie_dict.items()}

In [5]:
# movie_dict

In [6]:
for i in df_title.index:
    
    dates = df_title.at[i, 'Date']
    reg_date = re.sub("[.]|[ ]", "", dates)
    dt_list = []
    
    # Converting string date range to suitable `datetime` range
    
    for d1 in reg_date.split('–'):   
        d1 = re.sub("Sept", "Sep", d1)
        m1 = ''
        if len(d1) > 3:
            month = m1.join(re.findall("[a-zA-Z]", d1))
            day = m1.join(re.findall("[0-9]", d1))

            s = '{} {} {}'.format(month, day, df_title.at[i, 'Year'])
            dt_list.append(datetime.datetime.strptime(s, '%b %d %Y').date())
            
        else:
            s = '{} {} {}'.format(month, d1, df_title.at[i, 'Year'])
            dt_list.append(datetime.datetime.strptime(s, '%b %d %Y').date())

    df_title.at[i, 'begin_date'] = dt_list[0]
    df_title.at[i, 'end_date'] = dt_list[1]
    
    # check if a weekend exists within the date range
    new_range = []
    default_range = range(0, 7)
    wk_range = range(dt_list[0].weekday(), dt_list[1].weekday())
    
    if wk_range.start > wk_range.stop:
        k = wk_range.start
        for _ in range(len(default_range)):
            new_range.append(default_range[k % len(default_range)])
            k+=1
    else:
        new_range = list(wk_range)
    
    df_title.at[i, 'has_weekend'] = 5 in new_range or 6 in new_range
    df_title.at[i, 'nth_week_number'] = dt_list[0].isocalendar()[1]
    df_title.at[i, 'movie_id_t'] = int(movie_dict[df_title.at[i, 'Title']])
    
    df_title.at[i, 'gross_to_date_float'] = float(re.sub('[^\d.]', '', df_title.at[i, 'Gross-to-Date']))
    try:
        df_title.at[i, 'weekend_gross_float'] = float(re.sub('[^\d.]', '', df_title.at[i, 'Weekend Gross']))
    except:
        df_title.at[i, 'weekend_gross_float'] = 0 # or pass fillna(0) prior
        

In [7]:
for i in df_sbuzz.index:
    df_sbuzz.at[i, 'movie_id_s'] = int(movie_dict[df_sbuzz.at[i, 'title']])
    df_sbuzz.at[i, 'nth_week_number'] = df_sbuzz.at[i, 'post_date'].isocalendar()[1]
    df_sbuzz.at[i, 'is_influencer'] = df_sbuzz.at[i, 'user_followers_count'] > 10000

In [8]:
analyzer = SentimentIntensityAnalyzer()
df_sbuzz['tweet_sentiment'] = pd.Series(map(lambda x: x['compound'], df_sbuzz['post_content'].apply(analyzer.polarity_scores)))

In [20]:
df_sbuzz[df_sbuzz['movie_id_s']==23].sort_values(by='post_date')

,ID,title,post_link,post_date_time,post_content,user_name,user_bio,user_followers_count,user_followee_count,user_post_count,country,state,language,post_date,post_time,movie_id_s,nth_week_number,is_influencer,tweet_sentiment
77782,23,sweet country,http://twitter.com/Rvolkie/status/104616936876...,2018-09-29T22:46:29.000+0000,@TaikaWaititi Sweet country.,Rvolkie,Sports and movie fanatic. MA Swinburne. My wif...,211,254,1354,AUS,Australian Capital Territory,en,2018-09-29,22:46:29,23.0,39.0,False,0.4588
77781,23,sweet country,http://twitter.com/JacobBrinkworth/status/1046...,2018-09-30T03:11:41.000+0000,I don't do scores BUT if I did:. . - Madeline'...,JacobBrinkworth,amateur filmmaker/critic/person . . mind my typos,209,665,3568,AUS,NaN,en,2018-09-30,03:11:41,23.0,39.0,False,-0.5467
77780,23,sweet country,http://twitter.com/jacster75/status/1048652103...,2018-10-06T19:11:59.000+0000,I'm watching Sweet Country (2018) https://t.co...,jacster75,"Bro in the streets, Ho in the sheets",80,353,3966,AUS,Victoria,en,2018-10-06,19:11:59,23.0,40.0,False,0.4588
77779,23,sweet country,http://twitter.com/olindamedia/status/10587894...,2018-11-03T18:34:20.000+0000,RT @Screen_Producer Among the top earners at t...,olindamedia,"Future thinker, documentary maker, storyteller...",312,1377,5515,AUS,Western Australia,en,2018-11-03,18:34:20,23.0,44.0,False,0.5859
77778,23,sweet country,http://twitter.com/JacobBrinkworth/status/1058...,2018-11-04T03:03:36.000+0000,1. Madeline's Madeline (one of the most moving...,JacobBrinkworth,amateur filmmaker/critic/person . . mind my typos,287,720,4260,AUS,NaN,en,2018-11-04,03:03:36,23.0,44.0,False,0.8542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77711,23,sweet country,http://twitter.com/JacobBrinkworth/status/1084...,2019-01-13T03:42:23.000+0000,RT @FakeRobHunter SWEET COUNTRY is a great fil...,JacobBrinkworth,amateur filmmaker/critic/person . . mind my typos,520,864,6038,AUS,NaN,en,2019-01-13,03:42:23,23.0,2.0,False,0.9087
77708,23,sweet country,http://twitter.com/TheCurbAU/status/1084340200...,2019-01-13T06:43:44.000+0000,@Gkap42 @JacobBrinkworth Here's my top ten. . ...,TheCurbAU,NaN,2597,2082,42534,AUS,NaN,en,2019-01-13,06:43:44,23.0,2.0,False,0.8658
77707,23,sweet country,http://twitter.com/eli3a_UwU/status/1084323937...,2019-01-13T05:39:07.000+0000,"RT @NITV In the lead up to 26 January, NITV wi...",eli3a_UwU,decolonised space,10,55,1195,AUS,Victoria,en,2019-01-13,05:39:07,23.0,2.0,False,0.8020
77706,23,sweet country,http://twitter.com/stingers90/status/108676912...,2019-01-19T23:35:24.000+0000,#sweetcountry by Warwick Thornton is an excell...,stingers90,NaN,182,694,3833,AUS,New South Wales,en,2019-01-19,23:35:24,23.0,3.0,False,0.7574


----

```df_title``` is the dataframe which contains information regarding the box office performance for 25 movies. In this phase the raw string data are converted to their respective formats and unique movie ids are introduced. ```nth_week_number``` (converts respective date to week number) is another feature that is generated to check for seasonalities. 

In [9]:
df_title.sample(3) 

,Title,Date,Year,Rank,Weekend Gross,Gross-to-Date,Week # of release,begin_date,end_date,has_weekend,nth_week_number,movie_id_t,gross_to_date_float,weekend_gross_float
165,dunkirk,Aug. 17–20,2017,3,"$879,569","$16,410,771",5,2017-08-17,2017-08-20,True,33.0,11.0,16410771.0,879569.0
115,wonder woman,Jun. 22–25,2017,4,"$1,651,459","$17,882,004",4,2017-06-22,2017-06-25,True,25.0,7.0,17882004.0,1651459.0
216,ladies in black,Nov. 1–4,2018,8,"$151,331","$8,132,478",7,2018-11-01,2018-11-04,True,44.0,22.0,8132478.0,151331.0


```df_sbuzz``` is the dataframe that holds information about twitter performance for the movies from the box office. Additional features such as if the user is an influencer (> 10k followers) and tweet sentiment are further introduced. The polarity of the sentitments range from [-1, +1] with -1 denoting a negative sentiment and +1 a positive sentiment.

In [10]:
df_sbuzz.sample(3)

,ID,title,post_link,post_date_time,post_content,user_name,user_bio,user_followers_count,user_followee_count,user_post_count,country,state,language,post_date,post_time,movie_id_s,nth_week_number,is_influencer,tweet_sentiment
23286,2,avengers infinity war,http://twitter.com/twiinskeletons/status/99405...,2018-05-09T03:41:37.000+0000,RT @MCU_Tweets A new #AvengersInfinityWar offi...,twiinskeletons,melbourne,1711,251,57556,AUS,Victoria,en,2018-05-09,03:41:37,5.0,19.0,False,0.5423
3641,1,beauty and the beast,http://twitter.com/alana_bleize/status/8378254...,2017-03-04T00:42:09.000+0000,I vote for #ArianaGrande & #JohnLegend “Beauty...,alana_bleize,Don't let yourself be a victim of your negativ...,45,360,77,AUS,Victoria,en,2017-03-04,00:42:09,1.0,9.0,False,0.0000
36839,5,wonder woman,http://twitter.com/lys_lilla/status/8695305865...,2017-05-30T12:27:01.000+0000,RT @wonderdianas 'why is wonder woman even so ...,lys_lilla,幸せ im a neet .. 💜 @joycestick_ 💜 dp by @jeyrablue,234,219,1672,AUS,Queensland,en,2017-05-30,12:27:01,7.0,22.0,False,0.0000


In [11]:
df_sbuzz.to_csv("syndata/Box Office/twitter_clean.csv", sep=",", index=False, date_format="%Y-%m-%d")
df_title.to_csv("syndata/Box Office/box_office_clean.csv", sep=",", index=False, date_format="%Y-%m-%d")

-----------